In [1]:
import pandas as pd
import sys, os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from crimebb import *

In [3]:
DATA_PATH="../data/"
CSV_PATH = f"{DATA_PATH}csv/"

### Database connection

In [4]:
config = openFile(f'{DATA_PATH}/db_config.json')
config_file = config['postgres_connection']
config_file

{'host': 'localhost',
 'username': 'postgres',
 'password': '123456',
 'port': 9200}

### Listing db sizes

In [5]:
import glob

In [6]:
db_path = [db_.split("\\")[-1] for db_ in glob.glob(f"{CSV_PATH}/*")]

print("DB Name\n", db_path,"\n")

DB Name
 ['antichat-2021-01-10', 'deutschland-im-deep-web-2021-01-10', 'dread-2021-01-10', 'envoy-forum-2021-01-10', 'garage4hackers-2021-01-10', 'raidforums-2021-01-10', 'runion-2021-01-10', 'safe-sky-hacks-2021-01-10', 'the-hub-2021-01-10', 'torum-2021-01-10'] 



In [7]:
tables_dict = {}

for db_name in db_path:
    db_tables = [db_.split("\\")[-1] for db_ in glob.glob(f"{CSV_PATH}/{db_name}/*.csv")]
    tables_dict[db_name] = db_tables.copy()
    print(db_name, db_tables)

antichat-2021-01-10 ['boards.csv', 'contracts.csv', 'members.csv', 'posts.csv', 'threads.csv', 'votes.csv']
deutschland-im-deep-web-2021-01-10 ['boards.csv', 'contracts.csv', 'members.csv', 'posts.csv', 'threads.csv', 'votes.csv']
dread-2021-01-10 ['boards.csv', 'contracts.csv', 'members.csv', 'posts.csv', 'threads.csv', 'votes.csv']
envoy-forum-2021-01-10 ['boards.csv', 'contracts.csv', 'members.csv', 'posts.csv', 'threads.csv', 'votes.csv']
garage4hackers-2021-01-10 ['boards.csv', 'contracts.csv', 'members.csv', 'posts.csv', 'threads.csv', 'votes.csv']
raidforums-2021-01-10 ['boards.csv', 'contracts.csv', 'members.csv', 'posts.csv', 'threads.csv', 'votes.csv']
runion-2021-01-10 ['boards.csv', 'contracts.csv', 'members.csv', 'posts.csv', 'threads.csv', 'votes.csv']
safe-sky-hacks-2021-01-10 ['boards.csv', 'contracts.csv', 'members.csv', 'posts.csv', 'threads.csv', 'votes.csv']
the-hub-2021-01-10 ['boards.csv', 'contracts.csv', 'members.csv', 'posts.csv', 'threads.csv', 'votes.csv']
to

### Listing tables

In [8]:
tables_dict

{'antichat-2021-01-10': ['boards.csv',
  'contracts.csv',
  'members.csv',
  'posts.csv',
  'threads.csv',
  'votes.csv'],
 'deutschland-im-deep-web-2021-01-10': ['boards.csv',
  'contracts.csv',
  'members.csv',
  'posts.csv',
  'threads.csv',
  'votes.csv'],
 'dread-2021-01-10': ['boards.csv',
  'contracts.csv',
  'members.csv',
  'posts.csv',
  'threads.csv',
  'votes.csv'],
 'envoy-forum-2021-01-10': ['boards.csv',
  'contracts.csv',
  'members.csv',
  'posts.csv',
  'threads.csv',
  'votes.csv'],
 'garage4hackers-2021-01-10': ['boards.csv',
  'contracts.csv',
  'members.csv',
  'posts.csv',
  'threads.csv',
  'votes.csv'],
 'raidforums-2021-01-10': ['boards.csv',
  'contracts.csv',
  'members.csv',
  'posts.csv',
  'threads.csv',
  'votes.csv'],
 'runion-2021-01-10': ['boards.csv',
  'contracts.csv',
  'members.csv',
  'posts.csv',
  'threads.csv',
  'votes.csv'],
 'safe-sky-hacks-2021-01-10': ['boards.csv',
  'contracts.csv',
  'members.csv',
  'posts.csv',
  'threads.csv',
  'vo

### Getting CSV files

In [9]:
for db_name in list_dbs:
    db_path = f"{CSV_PATH}{db_name}/"
    db_tables = [db_.split("\\")[-1] for db_ in glob.glob(f"{db_path}/*")]
    
    for cur_table in db_tables:
        print(cur_table)
        table_df = pd.read_csv(f"{db_path}{cur_table}", sep=",", low_memory=False)
        print(table_df)
    break
    

NameError: name 'list_dbs' is not defined

In [ ]:
db_tables